In [1]:
import pandas as pd
import datetime
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [2]:
tick = pd.read_csv('../input/FFF/FFF_tick_within_window.csv')

In [3]:
tick.columns

Index(['date', 'Date-Time(EST)', '#RIC', 'Price', 'Statement Release Time',
       'Press Conference Start Time', 'scripted_end', 'pc_end',
       'months_to_exp'],
      dtype='object')

In [4]:
tick['Timestamp']=tick['Date-Time(EST)'].apply(lambda x: datetime.datetime.timestamp(datetime.datetime.strptime(x[:-6],'%Y-%m-%d %H:%M:%S.%f')))

In [5]:
tick.head()

,date,Date-Time(EST),#RIC,Price,Statement Release Time,Press Conference Start Time,scripted_end,pc_end,months_to_exp,Timestamp
0,2012-04-25 00:00:00.000,2012-04-25 12:18:10.490077-04:00,FFK2,99.865,2012-04-25 12:30:00.000,2012-04-25 14:15:00.000,2012-04-25 14:19:35.640,2012-04-25 15:06:23.400,1,1.335371e+09
1,2012-04-25 00:00:00.000,2012-04-25 12:18:10.490077-04:00,FFK2,99.865,2012-04-25 12:30:00.000,2012-04-25 14:15:00.000,2012-04-25 14:19:35.640,2012-04-25 15:06:23.400,1,1.335371e+09
2,2012-04-25 00:00:00.000,2012-04-25 12:18:10.490077-04:00,FFK2,99.865,2012-04-25 12:30:00.000,2012-04-25 14:15:00.000,2012-04-25 14:19:35.640,2012-04-25 15:06:23.400,1,1.335371e+09
3,2012-04-25 00:00:00.000,2012-04-25 12:18:10.493973-04:00,FFK2,99.865,2012-04-25 12:30:00.000,2012-04-25 14:15:00.000,2012-04-25 14:19:35.640,2012-04-25 15:06:23.400,1,1.335371e+09
4,2012-04-25 00:00:00.000,2012-04-25 12:22:12.994806-04:00,FFK2,99.865,2012-04-25 12:30:00.000,2012-04-25 14:15:00.000,2012-04-25 14:19:35.640,2012-04-25 15:06:23.400,1,1.335371e+09


In [6]:
from utils import agg_to_kline

In [7]:
res = []
interval = 1
for i in tick.groupby('date'):
    a = agg_to_kline(i[1],interval)
    #fill empty timeslots
    t1 = datetime.datetime.timestamp(datetime.datetime.strptime(i[1]['Press Conference Start Time'].iloc[0],'%Y-%m-%d %H:%M:%S.%f'))
    t2 = datetime.datetime.timestamp(datetime.datetime.strptime(i[1]['pc_end'].iloc[0],'%Y-%m-%d %H:%M:%S.%f'))
    num_groups = int((t2 - t1)//interval)+1
    group_list = np.array([i for i in range(num_groups)])+t1//interval
    group = group_list[np.isin(group_list,a['group'].values,invert=True)]
    app = pd.DataFrame()
    app['group'] = group
    app['Num_Trades'] = 0
    app['OpenTime'] = app['group'].apply(lambda x: x*interval)  
    _ = pd.concat([a,app]).sort_values(by='group')
    pp = _[_['group']>=group_list[0]]
    pp = pp[pp['group']<=group_list[-1]]
    pp['Trade_Or_Not'] = pp['Num_Trades'] > 0
    pp['Date'] = i[0]
    res.append(pp[['OpenTime','Date','Num_Trades','Trade_Or_Not']].reset_index(drop=True))

In [10]:
import os
conferences=[]
file_list = []
directory = os.path.join("../input/map/")
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith("sent.csv"):
            if file[:8] in ['20200303','20200315','20220615']:
                continue
            file_list.append(file[:8])
            conferences.append(pd.read_csv(directory+file,names=['id', 'start_time', 'end_time', 'content']))

In [11]:
len(conferences)

53

In [12]:
p_conferences=[]
for i in conferences:
    temp = pd.DataFrame()
    temp['start_time'] = i['start_time'].apply(lambda x: np.round(x))
    temp['end_time'] = i['end_time'].apply(lambda x: np.round(x))
    temp['content'] = i['content']
    p_conferences.append(temp)

In [13]:
len(p_conferences)

53

In [14]:
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
def content_to_words(x):
    """This function removes punctuations and stems the words"""
    return [ps.stem(word) for word in tokenizer.tokenize(x)]

In [15]:
count = []
to_add = []
def sentence_to_instance(sent, word_per_sec=4):
    global count
    global to_add
    duration = sent['duration']
    if duration == 0:
        to_add = sent['content_list']
        return np.nan
    length = int(duration*word_per_sec)
    arry = np.array(['[QUIET]' for _ in range(length)],dtype='object')
    if to_add is []:
        content = sent['content_list']
    else:
        content = sent['content_list'] + to_add
        to_add = []
    start = max(length-len(content), 0)
    for i,j in enumerate(content):
        if i == length:
            count.append(len(content)-length)
            break
        group = duration - i//duration - 1
        index = i%word_per_sec
        position = group *word_per_sec + index
        arry[start+int(i)] = j
    return arry

In [16]:
word_speed=[]
for i in p_conferences:
    i['content_list'] = i['content'].apply(content_to_words)
    i['Num_words'] = i['content_list'].apply(len)
    i['duration']= i['end_time']-i['start_time']
    i['word_speed']=i['Num_words']/(i['duration']+1)
    i['token_list'] = i.apply(sentence_to_instance, 1)


In [17]:
x = np.concatenate(pd.concat(p_conferences)['content_list'].values)
unique_words = sorted(set(x))
len(unique_words)

5016

In [22]:
final = []
for i in p_conferences:
    i = i.dropna()
    final_ = np.concatenate(i['token_list'].values)
    final.append(final_.reshape(-1,4))

In [23]:
len(res),len(final)

(53, 53)

In [34]:
data = []
for i,j in zip(res, final):
    temp_d = pd.DataFrame()
    if len(i) != len(j):
        i=i[:-1]
    temp_d['OpenTime']=i['OpenTime']
    temp_d['Date']=i['Date']
    temp_d['Num_Trades']=i['Num_Trades']
    temp_d['Trade_Or_Not']=i['Trade_Or_Not']
    
    temp_w = pd.DataFrame(j,columns=['Word1','Word2','Word3','Word4'])
    temp = pd.concat([temp_d,temp_w],axis=1)
    data.append(temp)

In [36]:
final_raw = pd.concat(data).reset_index(drop=True)

In [37]:
final_raw['Trade_Or_Not'].value_counts(normalize=True)

False    0.916449
True     0.083551
Name: Trade_Or_Not, dtype: float64

In [38]:
final_raw.to_csv('../input/processed/word_per_sec_with_trades.csv',index=False)